## The main file to invoke My Virtual Moments application

#### 1. Prepare the LLM for question answering.
In this naive implementation, we want to first ensure that llama3 (or any other possible models) may respond to user requests well.